<a href="https://colab.research.google.com/github/Jaramillo35/Natural-Language-Processing/blob/main/MNA_NLP_actividad_semanas_7_y_8_LDA_y_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Arturo Jain (A01794992)
  *   José Ashamat Jaimes Saavedra (A01736690)
  *   Owen Jáuregui (A01638122)
  *   Ramiro Martin Jaramillo Romero (A01171251)

* **Número de Equipo:**

  * Equipo 80


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [1]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
# Descargamos las fabulas del proyecto Gutenberg
# Obtenemos 10 audios en especifico (ids) y las guardamos de forma local

import os
from urllib import request

# Lista de IDs a descargar
ids = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]

# Ahora usamos la ruta .m4b en Gutenberg
base_url = "https://www.gutenberg.org/files/21144/m4b/21144-{id:02d}.m4b"

# Creamos el directorio para guardar los audios
os.makedirs("audios", exist_ok=True)

# Descargamos los audios y los guardamos en el directorio "audios"
for i in ids:
    url = base_url.format(id=i)
    out_path = f"audios/fabula_{i:02d}.m4b"
    print(f"Descargando {url} → {out_path}")
    request.urlretrieve(url, out_path)


Descargando https://www.gutenberg.org/files/21144/m4b/21144-01.m4b → audios/fabula_01.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-04.m4b → audios/fabula_04.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-05.m4b → audios/fabula_05.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-06.m4b → audios/fabula_06.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-14.m4b → audios/fabula_14.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-22.m4b → audios/fabula_22.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-24.m4b → audios/fabula_24.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-25.m4b → audios/fabula_25.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-26.m4b → audios/fabula_26.m4b
Descargando https://www.gutenberg.org/files/21144/m4b/21144-27.m4b → audios/fabula_27.m4b


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
# Usamos pydub para cargar y manipular los audios
# Tambien usamos Whisper para transcribir los audios
from pydub import AudioSegment
import numpy as np
import re
from pathlib import Path
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 1) Encuentra todos los audios
audio_dir = Path("audios")
audio_paths = list(audio_dir.glob("*.m4b")) + list(audio_dir.glob("*.mp3"))

# 2) Carga el modelo y el procesador
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model     = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")

# Funcion para cargar el audio y lo normalizamos
def load_audio(path: Path, target_sr: int = 16_000):
    seg = AudioSegment.from_file(str(path))
    seg = seg.set_frame_rate(target_sr).set_channels(1)
    arr = np.array(seg.get_array_of_samples(), dtype=np.float32)
    if seg.sample_width == 2:
        arr /= 2**15
    return arr, target_sr

# Funcion para transcribir el audio
def transcribir(path: Path):
    # a) Carga y normaliza en memoria
    audio, sr = load_audio(path)
    # b) Deja que Processor ponga language/task y te devuelva todo
    inputs = processor(
        audio,
        sampling_rate=sr,
        return_tensors="pt",
        language="es",    # fuerza español
        task="transcribe" # no traducción
    )
    # c) Genera directamente con todos los inputs preparados
    tokens = model.generate(**inputs)
    # d) Decodifica y devuelve texto
    return processor.batch_decode(tokens, skip_special_tokens=True)[0]

# 3) Bucle principal
# Creamos un diccionario para guardar los textos de los audios
textos = {}
for ap in audio_paths:
    m = re.search(r"(\d+)", ap.stem)
    if not m:
        continue
    fid = int(m.group(1))
    print(f"Transcribiendo fábula {fid} desde {ap.name}…")
    textos[fid] = transcribir(ap)



Transcribiendo fábula 1 desde fabula_01.m4b…
Transcribiendo fábula 22 desde fabula_22.m4b…
Transcribiendo fábula 24 desde fabula_24.m4b…
Transcribiendo fábula 14 desde fabula_14.m4b…
Transcribiendo fábula 25 desde fabula_25.m4b…
Transcribiendo fábula 6 desde fabula_06.m4b…
Transcribiendo fábula 26 desde fabula_26.m4b…
Transcribiendo fábula 4 desde fabula_04.m4b…
Transcribiendo fábula 5 desde fabula_05.m4b…
Transcribiendo fábula 27 desde fabula_27.m4b…


Por qué Whisper:

* Multilingüe con alta calidad en español.
* Fácil integración con Transformers.

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

import re

def limpiar_encabezado_pie(texto):
    # Eliminamos todo hasta la primera ocurrencia de "fábula número"
    texto = re.sub(r"^.*?fábula número \d+\s*[:\-–]?", "", texto, flags=re.IGNORECASE)
    # Eliminamos desde "fin de la fábula" hasta el final
    texto = re.sub(r"fin de la fábula.*$", "", texto, flags=re.IGNORECASE|re.DOTALL)
    return texto.strip()

for i in textos:
    textos[i] = limpiar_encabezado_pie(textos[i])

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

# Funcion para limpiar el texto
# Convertimos el texto a minusculas
# Eliminamos caracteres especiales
# Eliminamos espacios en blanco
# Devolvemos el texto limpio
def limpieza_basica(texto):
    texto = texto.lower()
    texto = re.sub(r"[^a-záéíóúüñ\s]", " ", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

# Limpiamos el texto de los audios (las fábulas)
for i in textos:
    textos[i] = limpieza_basica(textos[i])

La única limpieza realizada en este caso fue mantener los caracteres alfabéticos y espacios, eliminando cualquier otro símbolo de puntuación que pudiera haber. Debido a que las herramientas que vamos a usar soportan letras acentuadas, no es necesario sustituirlas. Para evitar tener caracteres sobrantes, también eliminamos espacios innecesarios.

# **Ejercicio 4:**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Supongamos que `textos` es tu dict {id: texto_limpio}
# Ordenamos los ids y creamos una lista de documentos
ids = sorted(textos)
docs = [textos[i] for i in ids]

# 1) Vectorizamos (cada fábula es un “documento”)
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(docs)

# 2) Creamos un LDA con tantos tópicos como documentos
lda = LatentDirichletAllocation(n_components=len(docs), random_state=42)
lda.fit(dtm)

# 3) Extraemos las 20 palabras más relevantes de cada tópico
terms = vectorizer.get_feature_names_out()
for topic_idx, comp in enumerate(lda.components_):
    top_indices = comp.argsort()[-20:][::-1]
    keywords = [terms[i] for i in top_indices]
    print(f"Fábula {ids[topic_idx]:02d}: {', '.join(keywords[:10])}…")



Fábula 01: lobo, el, de, un, cabrito, la, casa, por, una, vio…
Fábula 04: el, perro, león, en, un, con, se, su, retrocedió, perseguía…
Fábula 05: en, que, lo, lobo, de, el, asno, entre, la, sus…
Fábula 06: cruz, carnicero, campanilla, el, lobo, en, la, cordero, templo, perro…
Fábula 14: sopo, las, fábulas, de, cruz, carnicero, campanilla, el, lobo, en…
Fábula 22: cruz, carnicero, campanilla, el, lobo, en, la, cordero, templo, perro…
Fábula 24: cruz, carnicero, campanilla, el, lobo, en, la, cordero, templo, perro…
Fábula 25: cruz, carnicero, campanilla, el, lobo, en, la, cordero, templo, perro…
Fábula 26: el, se, perro, la, de, un, almeja, al, trataba, tragó…
Fábula 27: cruz, carnicero, campanilla, el, lobo, en, la, cordero, templo, perro…


# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
# Usamos GPT para generar resumenes y subtemas de las fábulas

vectorizer = CountVectorizer()
dtm        = vectorizer.fit_transform(docs)
lda        = LatentDirichletAllocation(n_components=len(docs), random_state=42)
lda.fit(dtm)

terms = vectorizer.get_feature_names_out()

# Reconstruir palabras_clave como dict {fábula_id: [kw1, kw2, ...]}
palabras_clave = {}
for topic_idx, comp in enumerate(lda.components_):
    top20_idx = comp.argsort()[-20:][::-1]
    keywords  = [terms[i] for i in top20_idx]
    palabras_clave[ids[topic_idx]] = keywords

# 1) Parámetros de OpenAI
from openai import OpenAI
client = OpenAI(api_key="TÚ API KEY")

# 2) Modelo que vas a usar
MODEL_NAME = "gpt-4.1"

# 3) Inicializa el dict donde guardarás los resultados
resultados = {}

# 4) Función para invocar a GPT
def generar_resumen_y_subtemas(claves, model=MODEL_NAME):
    prompt = f"""
Eres un asistente que, dado este conjunto de palabras clave de una fábula de Esopo: {claves},
1) Genera un enunciado que resuma la historia en una sola frase.
2) Genera tres subtemas distintos (cada uno en frase) para discusión o moraleja.
Devuelve JSON con campos "resumen" y "subtemas" (lista de 3 strings).
"""
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return resp.choices[0].message.content

# 5) Llamada para cada fábula
import json

for fid, claves in palabras_clave.items():
    print(f"Procesando fábula {fid:02d} con {MODEL_NAME}…")
    resultados[fid] = generar_resumen_y_subtemas(claves)

# 6) Imprime todos los resultados formateados
for fid in sorted(resultados):
    print(f"\nFábula {fid:02d}:")
    data = json.loads(resultados[fid])
    print(json.dumps(data, indent=2, ensure_ascii=False))

Procesando fábula 01 con gpt-4.1…
Procesando fábula 04 con gpt-4.1…
Procesando fábula 05 con gpt-4.1…
Procesando fábula 06 con gpt-4.1…
Procesando fábula 14 con gpt-4.1…
Procesando fábula 22 con gpt-4.1…
Procesando fábula 24 con gpt-4.1…
Procesando fábula 25 con gpt-4.1…
Procesando fábula 26 con gpt-4.1…
Procesando fábula 27 con gpt-4.1…

Fábula 01:
{
  "resumen": "Un lobo intentó atacar a un cabrito que estaba seguro dentro de un corral, pero el cabrito aprovechó la protección para burlarse e insultar al lobo sin miedo.",
  "subtemas": [
    "El valor de la seguridad y la protección frente al peligro.",
    "La valentía puede depender de las circunstancias y no solo del carácter.",
    "No es prudente insultar o provocar a otros solo porque estamos en una posición segura."
  ]
}

Fábula 04:
{
  "resumen": "Un perro perseguía a un león rugiendo, pero, atemorizado cuando el león se volvió, retrocedió rápidamente y abandonó la persecución.",
  "subtemas": [
    "El valor real solo se pru

# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**


* #### **Conclusiones de la actividad audio-a-texto:**

1. La combinación de Whisper para transcripción de audio y GPT-4 para análisis de texto demostró ser muy efectiva. Whisper logró transcribir con precisión las fábulas en español, mientras que GPT-4 pudo generar resúmenes coherentes y extraer moralejas relevantes, mostrando cómo diferentes modelos de IA pueden trabajar en conjunto para procesar y analizar contenido. El uso de IA para la transcripción de audio es un concepto muy común en la actualidad. Su uso extenido en múltiples tipos de aplicaciones muestra su utilidad para la sociedad actual, y esta actividad permite observar la útilidad de la misma.

2. El uso de Latent Dirichlet Allocation (LDA) para extraer palabras clave de cada fábula proporcionó una base sólida para el análisis posterior. A pesar de que las fábulas son textos cortos, LDA logró identificar términos relevantes que capturaron la esencia de cada historia. El uso de esta herramienta podría ayudar de forma significativa a clasificar textos usando palabras clave que lo definan. En nuestros resultados podemos observar que también aparecen muchas palabras comunes sin valor agregado. Hacer una limpieza previa para eliminar estas palabras podría mejorar ampliamente los resultados.

# **Fin de la actividad LDA y LMM: audio-a-texto**